In [1]:
import random
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from scipy import stats
from tqdm import tqdm

class HillAnt():
    def __init__(self):
        self.reset()

    def reset(self):
        self.pos_x = 0
        self.pos_y = 0
        self.seconds = 0
    
    def step(self):
        direction = random.choice([1, -1])
        is_y = random.choice([True, False])
        if is_y:
            self.pos_y += direction * 10
        else:
            self.pos_x += direction * 10
        self.seconds += 1

    @property
    def coordinates(self):
        return [self.pos_x, self.pos_y]
    
    
    def is_in_square(self, length):
        def cond():
            if abs(self.coordinates[0]) >= length or abs(self.coordinates[1])>= length:
                return True
            return False
        return cond
    
    
    def is_in_line(self, m, c):
        def cond():
            if (m * self.coordinates[0]) + c == self.coordinates[1]:
                return True
            return False
        return cond
    
    def satisfy_equation(self, eq):
        def cond():
            if(eq(self.coordinates[0], self.coordinates[1])):
                return True
            return False
        return cond

    

    def sim(self, condition):
        def run():
            sim_result = 0
            while not condition():
                self.step()
            sim_result = self.seconds
            self.reset()
            return sim_result
        return run






In [2]:
def monteCarlo(sim, n):
    results = []
    for _ in  tqdm(range(0, n)):
        result = sim()
        results.append(result)
    return results

hillAnt = HillAnt()

n2 = 1000000
n = 1000000

def boundary3(x, y):
    return (((x-2.5)/30)**2) + (((y-2.5)/40)**2) > 1

def square_boundary(x, y):
    return abs(x) >= 20 or abs(y)>= 20

def line(x, y):
    return (-1*x) + 10 == y

def analyze(res):
    
    print(f'mean:  {np.mean(res)}')
    print(f'median: {np.median(res)}')
    print(f'mode: {stats.mode(res)}')

    print(f'Means Confidence Intervals: {st.t.interval(0.95, len(res)-1, loc=sum(res)/len(res), scale=st.sem(res))}')
    

In [3]:
res1 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(boundary3)), n)
analyze(res1)

res2 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(square_boundary)), n)
analyze(res2)

res3 = monteCarlo(hillAnt.sim(hillAnt.satisfy_equation(line)), n2)
analyze(res3)


  0%|          | 0/1000000 [00:00<?, ?it/s]

100%|██████████| 1000000/1000000 [00:31<00:00, 31733.08it/s]


mean:  13.984073
median: 11.0
mode: ModeResult(mode=array([7]), count=array([102432]))
Means Confidence Intervals: (13.965191902155285, 14.002954097844716)


100%|██████████| 1000000/1000000 [00:08<00:00, 115840.15it/s]


mean:  4.500965
median: 3.0
mode: ModeResult(mode=array([2]), count=array([250497]))
Means Confidence Intervals: (4.49532329978984, 4.50660670021016)


  3%|▎         | 26031/1000000 [06:20<32:01, 506.77it/s]  

In [ ]:
res1

In [ ]:
max(set(res2), key=res2.count)

In [ ]:
max(set(res4), key=res2.count)